In [5]:
import pandas as pd
import torch
import gpytorch
import pyarrow

In [3]:
df = pd.read_feather('birdie.feather')

In [4]:
df = df.drop(columns=['index', 'datetime'])

In [ ]:
xy = pd.read_csv('00_coordinates_start_end.csv')
xy = xy.assign(box = xy.index)

In [ ]:
# spatial coordinates presumably in meters
xy.Coord_X.max()-xy.Coord_X.min(), xy.Coord_Y.max()-xy.Coord_Y.min()

In [ ]:
dfxy = df.merge(xy[['box', 'Coord_X', 'Coord_Y']])

In [ ]:
# |(SM[1-9]_label)'

In [ ]:
temp_columns = dfxy.filter(regex='(Temp[1-9](_label)?)').columns

In [ ]:
gpin = dfxy.drop(columns=temp_columns).drop(columns=['box']).rename(columns={'Coord_X':'x', 'Coord_Y':'y', 'norm_time':'t',})

In [ ]:
sensorwise_cols = [["t","x","y", f"SM{i}", f"SM{i}_Depth", f"SM{i}_label"] for i in range(1,7)]

In [ ]:
m4D = pd.concat([gpin[cols].rename(columns={f'SM{s+1}':'m', f'SM{s+1}_Depth':'z', f'SM{s+1}_label':'label'}) 
           for s, cols in enumerate(sensorwise_cols[:2])], axis=0)
m4D = m4D.dropna() # (2076508 -> 1872822)

In [ ]:
# Take out deterministically identified outliers! -  Don't want them to contaminate the GP estimation
m4D = m4D.loc[~m4D.label.isin(['Auto:Spike','Auto:BattV', 'Auto:Range']), :]

In [ ]:
m4D.reset_index().to_feather('m4D.feather')

In [ ]:
m3Dtrain = m4D[(37000 < m4D.t) & (m4D.t < 37250) & (m4D.z==0.1)].drop(columns=['z'])
m3Dtest = m4D[(37250 < m4D.t) & (m4D.t < 37500) & (m4D.z==0.1)].drop(columns=['z'])

In [ ]:
train_x = torch.tensor(m3Dtrain[['t', 'x', 'y']].values, dtype=torch.float32)
train_y = torch.tensor(m3Dtrain[['m']].values, dtype=torch.float32).squeeze()

In [ ]:
test_x = torch.tensor(m3Dtest[['t', 'x', 'y']].values, dtype=torch.float32)
test_y = torch.tensor(m3Dtest[['m']].values, dtype=torch.float32).squeeze()

In [ ]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.GridInterpolationKernel(
                gpytorch.kernels.RBFKernel(), grid_size=grid_size, num_dims=3
            )
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

In [ ]:
training_iterations = 50
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [ ]:
def train():
    for i in range(training_iterations):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

In [ ]:
%time train()

In [ ]:
model.eval(); likelihood.eval()

In [ ]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))
    # pred_labels = observed_pred.mean